# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [4]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()
df = pd.DataFrame(data_json['items'])
df_rj = df[df['uf'] == 'RJ']
df_rj

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,38893,Município,Paty do Alferes,3303856,RJ,PVL02.007600/2019-05,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,3.512000e+06,0,1,30/10/2019
1,6390,Município,Cantagalo,3301108,RJ,None,Arquivado,17944.000335/2008-36,2008-04-10T03:00:00Z,Operação contratual interna,Caminho da Escola,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4.831600e+05,0,0,14/05/2008
2,8643,Município,Rio de Janeiro,3304557,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000816/2009-22,2011-11-17T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.500000e+08,1,0,23/12/2011
3,9096,Município,Cordeiro,3301504,RJ,None,Arquivado,17944.000902/2008-54,2008-06-16T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.267500e+05,0,0,15/09/2008
4,8050,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.000703/2010-61,2011-04-29T03:00:00Z,Operação contratual interna,PNAFM,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4.900000e+07,1,0,06/06/2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,11272,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.001398/2013-77,2013-11-25T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5.000000e+08,1,0,06/12/2013
256,27100,Município,Rio de Janeiro,3304557,RJ,PVL02.002429/2017-78,Arquivado por decurso de prazo,17944.103668/2017-15,2018-07-31T21:50:09Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.000000e+08,0,0,07/01/2019
257,12404,Município,Resende,3304201,RJ,None,Regularizado,17944.001679/2008-62,2013-07-12T03:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Ampla Energia e Serviços S/A,Real,4.397135e+06,1,0,26/07/2013
258,5689,Município,Rio Claro,3304409,RJ,None,Arquivado,17944.000173/2008-36,2008-04-10T03:00:00Z,Operação contratual interna,Caminho da Escola,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5.692600e+05,0,0,13/05/2008


In [14]:
# 2) Seu código aqui
status_counts = df_rj['status'].value_counts()[:3]
status_counts 

status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64

In [15]:
# 3) Seu código aqui
df_rj['ano_status'] = df_rj['data_status'].str.slice(6, 10)
df_rj['ano_status']

0      2019
1      2008
2      2011
3      2008
4      2011
       ... 
255    2013
256    2019
257    2013
258    2008
259    2007
Name: ano_status, Length: 260, dtype: object

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [78]:
#1) Seu código aqui
def consultar_pvl(uf, tipo_interessado):
    # Ajusta o parâmetro do tipo de interessado para a URL
    tipo_interessado_param = 'Estado' if tipo_interessado == 'Estado' else 'Munic%C3%ADpio'
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado_param}'
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        return df
    else:
        raise Exception(f'Erro na requisição: Status Code {response.status_code}')


In [79]:
# 2) Seu código aqui
df_mg_estado = consultar_pvl('MG', 'Estado')
solicitacoes_arquivadas_mg = df_mg_estado[df_mg_estado['status'] == 'Arquivado por decurso de prazo'].shape[0]

In [80]:
# 3) Seu código aqui
df_ba_municipio = consultar_pvl('BA', 'Município')
municipio_ba_com_mais_deferidos = df_ba_municipio[df_ba_municipio['status'] == 'Deferido']['interessado'].value_counts().idxmax()

In [81]:
# 4) Seu código aqui
df_ba_estado = consultar_pvl('BA', 'Estado')
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)

print(f"Quantidade de solicitações 'Arquivado por decurso de prazo' para MG: {solicitacoes_arquivadas_mg}")
print(f"Município da Bahia com mais solicitações deferidas: {municipio_ba_com_mais_deferidos}")

Quantidade de solicitações 'Arquivado por decurso de prazo' para MG: 1
Município da Bahia com mais solicitações deferidas: Luís Eduardo Magalhães
